In [1]:
import pandas as pd
import mysql.connector as connection

In [53]:
try:
    mydb = connection.connect(
        host="localhost", 
        database="webscraper_database",
        user="webscraper", 
        passwd="ackj8636&fkf",
        use_pure=True
    )
    query = "Select * from condos;"
    df = pd.read_sql(query,mydb)
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

2003: Can't connect to MySQL server on 'localhost:3306' (111 Connection refused)


In [54]:
df.set_index("id", inplace=True)

KeyError: "None of ['id'] are in the columns"

In [ ]:
len(df)

In [50]:
df

,willhaben_code,postal_code,district,price,commission_fee,size,room_count,price_per_m2,discovery_date,title,url,edit_date,address
id,,,,,,,,,,,,,
1,560368301,8010,Graz,990,0.0,85,4,11.6471,2022-05-15,b'*PROVISIONSFREI - hochwertig sanierte 4 Zimm...,b'https://www.willhaben.at/iad/immobilien/d/mi...,"15.05.2022, 15:42 Uhr","Jauerburggasse 8, 8010 Graz, Steiermark"
2,569494332,8020,Graz,749,0.0,62,2,12.0806,2022-05-15,b'| OHNE PROVISION | AUSGEZEICHNETE ANBINDUNG ...,b'https://www.willhaben.at/iad/immobilien/d/mi...,"15.05.2022, 15:48 Uhr","Babenbergerstraße, 8020 Graz, Steiermark"
3,544478086,8010,Jakomini,695,0.0,55,2,12.6364,2022-05-15,b'Provisionsfrei! Traumhafte neu sanierte 2-Zi...,b'https://www.willhaben.at/iad/immobilien/d/mi...,"15.05.2022, 15:50 Uhr","Klosterwiesgasse 48, 8010 Jakomini, Graz, Stei..."
4,567717836,8010,Geidorf,650,3.6,0,1,0.0000,2022-05-15,b'Toplage im Univiertel',b'https://www.willhaben.at/iad/immobilien/d/mi...,"15.05.2022, 15:50 Uhr","Liebiggasse, 8010 Geidorf, Graz, Steiermark"
5,569485170,8020,Graz,0,3.6,58,3,0.0000,2022-05-15,b'Zimmer in sonniger 2er WG',b'https://www.willhaben.at/iad/immobilien/d/mi...,"15.05.2022, 15:52 Uhr","Baiernstrasse, 8020 Graz, Steiermark"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,526902855,8051,Graz,589,0.0,0,2,0.0000,2022-05-15,"b'Platzwunder - P14 - 2 Zimmer auf 33,87qm mit...",b'https://www.willhaben.at/iad/immobilien/d/mi...,"13.05.2022, 10:35 Uhr","Anton-Kleinoscheg-Straße 66C, 8051 Graz, Steie..."
367,568914476,8010,Graz,978,0.0,95,3,10.2947,2022-05-15,b'Zentral gelegene 2er WG mit Wintergarten - P...,b'https://www.willhaben.at/iad/immobilien/d/mi...,"13.05.2022, 10:36 Uhr","Grazbachgasse, 8010 Graz, Steiermark"
368,568914631,8020,Graz,664,0.0,54,2,12.2963,2022-05-15,b'Genie\xc3\x9fen Sie das urbane Grazer Stadtl...,b'https://www.willhaben.at/iad/immobilien/d/mi...,"13.05.2022, 10:52 Uhr","8020 Graz, Steiermark"


In [51]:
df.groupby("postal_code").price.mean().sort_values()

postal_code
8055     521.857143
8010     541.397849
8045     545.000000
8051     582.416667
8042     595.666667
8053     602.823529
8054     623.677419
8041     632.285714
8052     646.416667
8020     655.006667
8047     761.250000
8043     865.000000
8044    1150.000000
Name: price, dtype: float64

In [52]:
df.groupby("postal_code").size().sort_values()

postal_code
8044      2
8043      4
8047      4
8041      7
8045      8
8042      9
8051     12
8052     12
8053     17
8055     21
8054     31
8010     93
8020    150
dtype: int64

In [22]:
import geopandas as gpd

In [31]:
gdf = gpd.read_file("graz.kml")

ValueError: Null layer: ''